#Inicializations

In [ ]:
BASEDIR = '.'

TRAINDATAPATH = 'aug_dataset_covid/train'

TESTDATAPATH = 'aug_dataset_covid/test'


VALIDDATAPATH = 'aug_dataset_covid/valid'

NEWTESTDATAPATH = 'dataset_covid/testNew'

Make necessary imports

In [ ]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,\
                                    MaxPooling2D,BatchNormalization,\
                                    MaxPooling2D,AveragePooling2D,\
                                    GlobalMaxPooling2D,GlobalAveragePooling2D,\
                                    Concatenate,Input,ZeroPadding2D,Reshape
                                    
from keras.layers.merge import concatenate
import numpy as np
import time
import os

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.callbacks import TensorBoard

from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model

from tensorflow.keras.applications.resnet import preprocess_input


import matplotlib.pyplot as plt

In [ ]:
#Corre
import matplotlib.pyplot as plt


def plot_acc_loss(result, epochs):
    acc = result.history['accuracy']
    loss = result.history['loss']
    val_acc = result.history['val_accuracy']
    val_loss = result.history['val_loss']
    plt.figure(figsize=(15, 5))
    plt.subplot(121)
    plt.plot(range(1,epochs), acc[1:], label='Train_acc')
    plt.plot(range(1,epochs), val_acc[1:], label='Test_acc')
    plt.title('Accuracy over ' + str(epochs) + ' Epochs', size=15)
    plt.legend()
    plt.grid(True)
    plt.subplot(122)
    plt.plot(range(1,epochs), loss[1:], label='Train_loss')
    plt.plot(range(1,epochs), val_loss[1:], label='Test_loss')
    plt.title('Loss over ' + str(epochs) + ' Epochs', size=15)
    plt.legend()
    plt.grid(True)
    plt.show()



#Dataset


**Load for the first time the dataset**

We first begin by defining the preprocesses that our data is going to suffer, in this case it suffers the recomended preprocess by RESNET as well as the possibility of beeing introduced some small alterations on the original image so our model generalzes better during training (small rotations, shears, zooming and mirroring)


We make use of the flow_from_directory method available to the ImageDataGenerator object we created. This function will look at the given directory and separate the images in 2 classes based on our subdirectories.

The images are given on 224 by 224, since its the recomended size to feed to a ResNet model.

Since the generator is an iterator, the batch size decides how many elements are to be "returned" on each next() call.

We also shuffle the data to avoid situations where the model receives to much of the same consecutive class, and ends up not generalizing but instead memorizing.


**Note:** the validation is composed of 20% of the original training dataset

In [ ]:

train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,rotation_range=30,shear_range=0.25,zoom_range=0.1, horizontal_flip = True) 
train_generator=train_datagen.flow_from_directory(TRAINDATAPATH, 
                                                 target_size=(224,224),
                                                 batch_size=1,
                                                 class_mode='binary',
                                                 shuffle=True)

print(type(train_generator))


valid_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,rotation_range=30,shear_range=0.25,zoom_range=0.1, horizontal_flip = True) 
valid_generator=valid_datagen.flow_from_directory(VALIDDATAPATH, 
                                                 target_size=(224,224),
                                                 batch_size=1,
                                                 class_mode='binary',
                                                 shuffle=True)

See what numbers have been atributed to each class

In [ ]:
print(train_generator.class_indices)

Visualize the data

Note that the images have already been processed by the resnet preprocessor, thus the color pallet and the image slight alterations

In [ ]:
x, y= next(train_generator)
image = x_batch[0]
plt.imshow(image,)
plt.show()

eval_generator.reset()

#ResNet50


Import **ResNet50Model**![alt text](https://miro.medium.com/max/2000/1*zbDxCB-0QDAc4oUGVtg3xw.png) 

Remove output layer from the base model, and get the weights of this networks trained for imagenet

In [ ]:
base_model=ResNet50(include_top=False, weights='imagenet')

Add our own layers to the end of the model 

In [ ]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(512,activation='relu')(x) 
x=Dropout(0.5)(x) 
preds=Dense(1,activation='sigmoid')(x)

model=Model(inputs=base_model.input,outputs=preds)

Freeze the weights on all layers except ours

In [ ]:
for layer in model.layers[:-4]:
    layer.trainable=False
for layer in model.layers[-4:]:
    layer.trainable=True
    

Compile the model using the Adam Optimizer, and the binary loss entropy function since our problem is a binary classification one.

The steps sizes are calculated given the total number of images and the batch sizes of the correspondent generators

We also chose 20 epochs for this first test

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size
model.fit_generator(train_generator,
                    steps_per_epoch =step_size_train,
                    validation_data = valid_generator,
                    validation_steps = step_size_valid,
                    epochs= 20,
                    )
 

Now that we verified that some results are obtained from the last train, we proceed to find the best hyper parameters:

The followed parameters where testes for the following values:

Number of dense layers : 1, 2, 3, 4

Number of nodes per dense layer: 16, 32, 64, 128, 256

Value of dropout: 0, 0.2, 0.5

Batch Size: 1, 2, 3

Number of epochs trained: 10. 20. 30





In [ ]:
from tensorflow.keras.applications.resnet import preprocess_input
import os
from tensorflow.keras import optimizers

NUMBER_DENSE = [2,3,4]
DENSE_NODES = [16,32,64]
DROPOUT = [0]
BATCH_SIZE = [2]
EPOCHS = [20]


for denseNumber in NUMBER_DENSE:
  for denseNodes in DENSE_NODES:
    for dropoutNumber in DROPOUT:
      for batchSize in BATCH_SIZE:
        for epochNumber in EPOCHS:
        

          LOGNAME = "RESNET50_RUN2.0_DenseLayers-{0}_DenseNodes-{1}_Dropout-{2}_BatchSize-{3}_EpochNumber-{4}".format(denseNumber,denseNodes,dropoutNumber,batchSize,epochNumber)
          log_dir = os.path.join(BASEDIR,"logs",LOGNAME,"fit",)
          tensorboard = TensorBoard(log_dir = log_dir)
          print(LOGNAME)

          train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,rotation_range=30,shear_range=0.25,zoom_range=0.1, horizontal_flip = True) 
          train_generator=train_datagen.flow_from_directory(TRAINDATAPATH, 
                                                          target_size=(224,224),
                                                          batch_size=batchSize,
                                                          class_mode='binary',
                                                          shuffle=True)


          valid_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,rotation_range=30,shear_range=0.25,zoom_range=0.1, horizontal_flip = True) 
          valid_generator=valid_datagen.flow_from_directory(VALIDDATAPATH, 
                                                          target_size=(224,224),
                                                          batch_size=batchSize,
                                                          class_mode='binary',
                                                          shuffle=True)

          nOfNew = -2 
          base_model=ResNet50(include_top=False, weights='imagenet')
          x=base_model.output
          x=GlobalAveragePooling2D()(x)

          for i in range(denseNumber-1):
            x=Dense(denseNodes,activation='relu')(x) 
            x=Dropout(dropoutNumber)(x) 
            nOfNew-=2

          preds=Dense(1,activation='sigmoid')(x)

          model=Model(inputs=base_model.input,outputs=preds)


          for layer in model.layers[:nOfNew]:
              layer.trainable=False
          for layer in model.layers[nOfNew:]:
              layer.trainable=True
              

          model.compile(loss='binary_crossentropy',optimizer=optimizers.Adam(lr=0.0005),metrics=['accuracy'])

          step_size_train=train_generator.n//train_generator.batch_size
          step_size_valid=valid_generator.n//valid_generator.batch_size
          model.fit_generator(train_generator,
                              steps_per_epoch =step_size_train,
                              validation_data = valid_generator,
                              validation_steps = step_size_valid,
                              epochs= epochNumber,
                              callbacks=[tensorboard]
                              )



After this intensive search the best results where found looking to the graphs given by the TensorBoard platform (and saved in the logs directory)


**Train accuracy:**

![alt text](https://cdn.discordapp.com/attachments/498949663513116673/698239341406978048/unknown.png)

**Train loss:**

![alt text](https://cdn.discordapp.com/attachments/498949663513116673/698239373052739615/unknown.png)




**Validation accuracy:**

![alt text](https://cdn.discordapp.com/attachments/498949663513116673/698239220145455115/unknown.png)


**Validation loss:**

![alt text](https://cdn.discordapp.com/attachments/498949663513116673/698239373052739615/unknown.png)



We then verified that the best results where given for the following parameters:

Number of dense layers: 2 and 3 (whe ended up chosing 2 for training time reasons since there was no noticable difference in preformace with2 or 3 layers)

Number of nodes per dense layer: 32 or 64 (similarly to the number of layers the diference between this numbers werent noticable we opted to chose based on the one that reduced the training time - 32)

The batch size number variation didnt seem to affect at all: so we ended up chosing 2

The dropout inclusion gave worst results than the ones without so we ended up not using one:






Bellow is the final model for training:

In [ ]:
from tensorflow.keras.applications.resnet import preprocess_input
import os
from tensorflow.keras import optimizers

train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,rotation_range=30,shear_range=0.25,zoom_range=0.1, horizontal_flip = True) 
train_generator=train_datagen.flow_from_directory(TRAINDATAPATH, 
                                                target_size=(224,224),
                                                batch_size=1,
                                                class_mode='binary',
                                                shuffle=True)


valid_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,rotation_range=30,shear_range=0.25,zoom_range=0.1, horizontal_flip = True) 
valid_generator=valid_datagen.flow_from_directory(VALIDDATAPATH, 
                                                target_size=(224,224),
                                                batch_size=1,
                                                class_mode='binary',
                                                shuffle=True)

 
base_model=ResNet50(include_top=False, weights='imagenet')
x=base_model.output
x=GlobalAveragePooling2D()(x)


x=Dense(32,activation='relu')(x) 
x=Dense(32,activation='relu')(x) 


preds=Dense(1,activation='sigmoid')(x)

model=Model(inputs=base_model.input,outputs=preds)


for layer in model.layers[:-4]:
    layer.trainable=False
for layer in model.layers[-4:]:
    layer.trainable=True
    

model.compile(loss='binary_crossentropy',optimizer=optimizers.Adam(lr=0.0005),metrics=['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.TruePositives(),  tf.keras.metrics.TrueNegatives(),  tf.keras.metrics.FalsePositives(),  tf.keras.metrics.FalseNegatives()])

model.summary()
step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size
history=model.fit_generator(train_generator,
                    steps_per_epoch =step_size_train,
                    validation_data = valid_generator,
                    validation_steps = step_size_valid,
                    epochs= 20,
                    )

In [ ]:
#Corre
plot_acc_loss(history, 20)

We proceed to test the model against the test set (20% of the original dataset)

In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

eval_generator = test_datagen.flow_from_directory(
        TESTDATAPATH,
        target_size=(224,224),
        batch_size=1,
        shuffle=False,
        seed=42,
        class_mode="binary")



eval_generator.reset()

In [ ]:
x = model.evaluate_generator(eval_generator,
                           steps = np.ceil(len(eval_generator) / 1),
                           use_multiprocessing = False,
                           verbose = 1,
                           workers=1
                           )


print('Test loss:' , x[0])
print('Test accuracy:',x[1])
print('Test precision:' , x[2])
print('Test Recall:',x[3])
print('Test F1 Score: ',2*(x[2]*x[3])/(x[2]+x[3]))

We finalize by visualizing the predictions of th model

In [ ]:
eval_generator.reset()  
pred = model.predict_generator(eval_generator,steps = np.ceil(len(eval_generator) / 1),verbose=1)
print("Predictions finished")

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
trueP= x[4]
trueN = x[5]
falseP = x[6]
falseN = x[7]
array = [[trueP,falseP],
         [falseN,trueN]]

array = [[trueP,falseP], [falseN,trueN]]
df_cm = pd.DataFrame(array, ["non-covid19","covid19"], ["non-covid19","covid19"])
sn.set(font_scale=1.4)

sn.heatmap(df_cm, annot=True, annot_kws={"size": 16},cmap="Blues") 

In [ ]:
import cv2

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

for index, probability in enumerate(pred):
    image_path = TESTDATAPATH + "/" +eval_generator.filenames[index]
    image = mpimg.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    pixels = np.array(image)
    plt.imshow(pixels)
    
    print(eval_generator.filenames[index])
    if probability > 0.5:
        plt.title("%.2f" % (probability[0]*100) + "% Non-COVID19")
    else:
        plt.title("%.2f" % ((1-probability[0])*100) + "% COVID19")
    plt.show()